In [1]:
import torch
import spacy
import numpy as np
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from torchtext import data

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tweet_data = load_dataset("SetFit/tweet_sentiment_extraction")
tweet_data_train = tweet_data['train']
tweet_data_test = tweet_data['test']

Using custom data configuration SetFit--tweet_sentiment_extraction-bee4b4571daa6a0e
Reusing dataset json (C:\Users\20172613\.cache\huggingface\datasets\json\SetFit--tweet_sentiment_extraction-bee4b4571daa6a0e\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
from sklearn.model_selection import train_test_split
train_split, valid_test = train_test_split(tweet_data_train, test_size=0.2, random_state=42)
tweet_data_train[0]['text']


datasets.arrow_dataset.Dataset

In [3]:
#tweet_data['train'][np.random.randint(0, len(tweet_data['train']))]

{'textID': '708b41b0c8',
 'text': ' I don`t think I am, my sisters refusn to get me a ticket now.  what you doing next week?',
 'label': 0,
 'label_text': 'negative'}

In [52]:
sample_text = 'NO MORE MSHS!!!!!!!! Gotta go to work...too tired'
tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
sample_tokenized = tokenizer(sample_text)

def yield_tokens(data_iter):
    for t in data_iter['text']:
        yield tokenizer(t)

vocab = build_vocab_from_iterator(yield_tokens(tweet_data_train))
print(f"vocab size: {len(vocab)}")
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: x



vocab size: 34096


In [51]:
len(vocab)

34096

In [49]:
vocab.freqs.most_common(20)

AttributeError: 'Vocab' object has no attribute 'freqs'

In [ ]:
def collate_bat